## Data Import

In [1]:
import pandas as pd
import csv

all_tweets = pd.read_csv('/home/muskan_2117/Downloads/SemEval2017_train_data.csv')
all_tweets.shape


(50248, 4)

In [2]:
all_tweets.head(10)

,id,label,text,Unnamed: 3
0,2.600000e+17,neutral,"Won the match #getin . Plus, tomorrow is a ver...",NaN
1,2.640000e+17,neutral,Some areas of New England could see the first ...,NaN
2,2.640000e+17,negative,Not Available,NaN
3,2.640000e+17,neutral,Not Available,NaN
4,2.640000e+17,neutral,Not Available,NaN
5,2.640000e+17,positive,Not Available,NaN
6,2.630000e+17,positive,@mariakaykay aga tayo tomorrow ah. :) Good nig...,NaN
7,2.600000e+17,neutral,Tina Fey &amp; Amy Poehler are hosting the Gol...,NaN
8,2.630000e+17,positive,Lunch from my new Lil spot ...THE COTTON BOWL ...,NaN
9,2.640000e+17,positive,SNC Halloween Pr. Pumped. Let's work it for Su...,NaN


## Using text pre-processing tool 'ekphrasis'

In [3]:
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
        'time', 'url', 'date', 'number'],
    # terms that will be annotated
    annotate={"hashtag", "allcaps", "elongated", "repeated",
        'emphasis', 'censored'},
    fix_html=True,  # fix HTML tokens
    
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter", 
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=False,  # spell correction for elongated words
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)

/home/muskan_2117/anaconda3/lib/python3.7/site-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Reading twitter - 1grams ...
Reading twitter - 2grams ...


/home/muskan_2117/anaconda3/lib/python3.7/site-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


Reading twitter - 1grams ...


In [4]:
tokens = []
NaNCount = 0

for i in range(len(all_tweets)):
    if all_tweets.text[i] == 'Not Available':
        tokens.append(['NaN'])
        NaNCount = NaNCount + 1
    else:
        tmp = text_processor.pre_process_doc(all_tweets.text[i])
        tokens.append(tmp)   

In [5]:
all_tweets['tokens'] = tokens 

In [6]:
all_tweets.head(10)

,id,label,text,Unnamed: 3,tokens
0,2.600000e+17,neutral,"Won the match #getin . Plus, tomorrow is a ver...",NaN,"[won, the, match, <hashtag>, get, in, </hashta..."
1,2.640000e+17,neutral,Some areas of New England could see the first ...,NaN,"[some, areas, of, new, england, could, see, th..."
2,2.640000e+17,negative,Not Available,NaN,[NaN]
3,2.640000e+17,neutral,Not Available,NaN,[NaN]
4,2.640000e+17,neutral,Not Available,NaN,[NaN]
5,2.640000e+17,positive,Not Available,NaN,[NaN]
6,2.630000e+17,positive,@mariakaykay aga tayo tomorrow ah. :) Good nig...,NaN,"[<user>, aga, tayo, tomorrow, ah, ., <happy>, ..."
7,2.600000e+17,neutral,Tina Fey &amp; Amy Poehler are hosting the Gol...,NaN,"[tina, fey, &, amy, poehler, are, hosting, the..."
8,2.630000e+17,positive,Lunch from my new Lil spot ...THE COTTON BOWL ...,NaN,"[lunch, from, my, new, lil, spot, ., <repeated..."
9,2.640000e+17,positive,SNC Halloween Pr. Pumped. Let's work it for Su...,NaN,"[<allcaps>, snc, </allcaps>, halloween, pr, .,..."


## Removal of 'Not Available' tweets from the data

In [7]:
import numpy as np

data = np.array([np.arange(len(all_tweets) - NaNCount)]*2).T
tweets = pd.DataFrame(data, columns = ['label','tokens']) 

j = 0

for i in range(len(all_tweets)):
    if all_tweets.tokens[i] == ['NaN']:
        continue
    else:
        tweets['label'][j] = all_tweets['label'][i]
        tweets['tokens'][j] = all_tweets['tokens'][i]
        j = j + 1

/home/muskan_2117/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/home/muskan_2117/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [8]:
print('total tweets', len(all_tweets))
print('NaNCount', NaNCount)
print('available tweets', len(tweets))

total tweets 50248
NaNCount 8313
available tweets 41935


In [9]:
tweets.head(10)

,label,tokens
0,neutral,"[won, the, match, <hashtag>, get, in, </hashta..."
1,neutral,"[some, areas, of, new, england, could, see, th..."
2,positive,"[<user>, aga, tayo, tomorrow, ah, ., <happy>, ..."
3,neutral,"[tina, fey, &, amy, poehler, are, hosting, the..."
4,positive,"[lunch, from, my, new, lil, spot, ., <repeated..."
5,positive,"[<allcaps>, snc, </allcaps>, halloween, pr, .,..."
6,negative,"[<user>, i, am, sorry, ,, i, heart, paris, is,..."
7,neutral,"[manchester, united, will, try, to, return, to..."
8,neutral,"[going, to, a, bulls, game, with, aaliyah, &, ..."
9,neutral,"[any, toon, fans, with, a, spare, ticket, for,..."


In [10]:
tweets.shape

(41935, 2)

## Using pre-trained GloVe word embeddings

In [ ]:
import gensim

model = gensim.models.KeyedVectors.load_word2vec_format('/home/muskan_2117/DAIICT/Sem2/IT550/project_datasets/datastories-semeval2017-task4-master/embeddings/glove.txt', binary=False)

In [7]:
model_w2v = gensim.models.KeyedVectors.load_word2vec_format('/home/muskan_2117/DAIICT/Sem2/IT550/project_datasets/datastories-semeval2017-task4-master/embeddings/glove.txt', binary=False)

In [8]:
all_embeddings = []

for i in range(len(tweets)):
    token = tweets['tokens'][i]
    embeddings = []
    for j in range(len(token)):
        try:
            embeddings.append(model[token[j]])
        except KeyError:
            continue
    all_embeddings.append(embeddings)
            

In [9]:
def word_vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0
    for word in tokens:
        try:
            vec += model_w2v[word].reshape((1, size))
            count += 1
        except KeyError:   # handling the case where the token is not in vocabulary
            continue
            
    if count != 0:
        vec /= count
    return vec

In [10]:
wordvec_arrays = np.zeros((len(tweets), 300))

for i in range(len(tweets)):
    wordvec_arrays[i,:] = word_vector(tweets['tokens'][i], 300)
    
wordvec_df = pd.DataFrame(wordvec_arrays)
wordvec_df.shape

(41935, 300)

In [79]:
len(all_embeddings)

41935

In [12]:
tweets['glove_embeddings'] = all_embeddings

In [81]:
tweets.head(10)

,label,tokens,glove_embeddings
0,neutral,"[won, the, match, <hashtag>, get, in, </hashta...","[[-0.102421, -0.095505, 0.245394, -0.279824, -..."
1,neutral,"[some, areas, of, new, england, could, see, th...","[[-0.064235, 0.06055, 0.176825, -0.460392, -1...."
2,positive,"[<user>, aga, tayo, tomorrow, ah, ., <happy>, ...","[[-0.588336, 0.265917, 0.95253, 0.381975, 0.33..."
3,neutral,"[tina, fey, &, amy, poehler, are, hosting, the...","[[0.135389, -0.255092, -0.065636, -0.166235, 0..."
4,positive,"[lunch, from, my, new, lil, spot, ., <repeated...","[[0.199795, -0.165797, 0.788851, -0.299232, 0...."
5,positive,"[<allcaps>, snc, </allcaps>, halloween, pr, .,...","[[0.876378, -0.987462, -0.733877, -2.11284, -1..."
6,negative,"[<user>, i, am, sorry, ,, i, heart, paris, is,...","[[-0.588336, 0.265917, 0.95253, 0.381975, 0.33..."
7,neutral,"[manchester, united, will, try, to, return, to...","[[0.262762, -0.669081, 0.019064, 0.117824, 0.7..."
8,neutral,"[going, to, a, bulls, game, with, aaliyah, &, ...","[[0.169069, 0.392756, 0.091623, -0.193693, 0.1..."
9,neutral,"[any, toon, fans, with, a, spare, ticket, for,...","[[0.461138, 0.068601, -0.367184, -0.320787, -0..."


## Test and train set

In [13]:
from sklearn.model_selection import train_test_split

#X = tweets.glove_embeddings
X = tweets['glove_embeddings']
#X = X.reshape(-1, 1)
y = tweets.label
#y = np.array(tweets['label'])
#y = y.reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(wordvec_df, y, test_size=0.11, random_state=42)

In [83]:
X_train.shape

(37322, 300)

In [84]:
X_test.shape

(4613, 300)

### Using Naive Bayes Classifier

In [14]:
from sklearn.metrics import classification_report, confusion_matrix

labels = ['positive', 'neutral', 'negative']

In [64]:
nb_embeddings = []

for i in range(len(tweets)):
    token = tweets['tokens'][i]
    embeddings = []
    for j in range(len(token)):
        try:
            embd = model[token[j]]
            minimum = np.amin(embd)
            maximum = np.amax(embd)
            tmp = np.array(embd)
            for j in range(len(tmp)):
                tmp[j] = (tmp[j] - minimum)/(maximum - minimum)    
            embeddings.append(tmp)
        except KeyError:
            continue
    nb_embeddings.append(embeddings)
            

In [65]:
tweets['nb_embeddings'] = nb_embeddings

In [66]:
tweets.head(10)

,label,tokens,glove_embeddings,nb_embeddings
0,neutral,"[won, the, match, <hashtag>, get, in, </hashta...","[[-0.102421, -0.095505, 0.245394, -0.279824, -...","[[0.5127671, 0.5152366, 0.6369606, 0.4494222, ..."
1,neutral,"[some, areas, of, new, england, could, see, th...","[[-0.064235, 0.06055, 0.176825, -0.460392, -1....","[[0.4912498, 0.5357633, 0.57724124, 0.34993187..."
2,positive,"[<user>, aga, tayo, tomorrow, ah, ., <happy>, ...","[[-0.588336, 0.265917, 0.95253, 0.381975, 0.33...","[[0.485613, 0.58785176, 0.670027, 0.60174185, ..."
3,neutral,"[tina, fey, &, amy, poehler, are, hosting, the...","[[0.135389, -0.255092, -0.065636, -0.166235, 0...","[[0.60720074, 0.3801686, 0.49032146, 0.4318314..."
4,positive,"[lunch, from, my, new, lil, spot, ., <repeated...","[[0.199795, -0.165797, 0.788851, -0.299232, 0....","[[0.55579877, 0.4088209, 0.79261523, 0.3551764..."
5,positive,"[<allcaps>, snc, </allcaps>, halloween, pr, .,...","[[0.876378, -0.987462, -0.733877, -2.11284, -1...","[[0.54221195, 0.26383027, 0.30170554, 0.095744..."
6,negative,"[<user>, i, am, sorry, ,, i, heart, paris, is,...","[[-0.588336, 0.265917, 0.95253, 0.381975, 0.33...","[[0.485613, 0.58785176, 0.670027, 0.60174185, ..."
7,neutral,"[manchester, united, will, try, to, return, to...","[[0.262762, -0.669081, 0.019064, 0.117824, 0.7...","[[0.5813098, 0.22103326, 0.48708934, 0.5252727..."
8,neutral,"[going, to, a, bulls, game, with, aaliyah, &, ...","[[0.169069, 0.392756, 0.091623, -0.193693, 0.1...","[[0.5443364, 0.6200081, 0.5181369, 0.42161644,..."
9,neutral,"[any, toon, fans, with, a, spare, ticket, for,...","[[0.461138, 0.068601, -0.367184, -0.320787, -0...","[[0.7392031, 0.59675884, 0.43862063, 0.4554572..."


In [99]:
def nb_word_vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0
    global_minimum = 0
    for word in tokens: 
        try:
            embd = model_w2v[word]
            #embd = model[token[j]]
            minimum = np.amin(embd)
            if minimum < global_minimum:
                global_minimum = minimum
        except KeyError:   # handling the case where the token is not in vocabulary
            continue
      
    for word in tokens: 
        try:
            #maximum = np.amax(embd)
            tmp = np.array(embd)
            for j in range(len(tmp)):
                tmp[j] = tmp[j] + abs(global_minimum)
            vec += tmp.reshape((1, size))
            count += 1
        except KeyError:   # handling the case where the token is not in vocabulary
            continue
            
    if count != 0:
        vec /= count
    return vec

In [100]:
nb_wordvec_arrays = np.zeros((len(tweets), 300))

for i in range(len(tweets)):
    nb_wordvec_arrays[i,:] = nb_word_vector(tweets['tokens'][i], 300)
    
nb_wordvec_df = pd.DataFrame(nb_wordvec_arrays)
nb_wordvec_df.shape

(41935, 300)

In [101]:
y_nb = tweets.label

X_train_nb, X_test_nb, y_train_nb, y_test_nb = train_test_split(nb_wordvec_df, y_nb, test_size=0.11, random_state=42)

In [89]:
X_train_nb.shape

(37322, 300)

In [90]:
X_test_nb.shape

(4613, 300)

In [102]:
from sklearn.naive_bayes import ComplementNB

clf = ComplementNB()

clf.fit(X_train_nb, y_train_nb)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred, target_names=labels))

print(confusion_matrix(y_test, y_pred, labels))

              precision    recall  f1-score   support

    positive       0.24      0.10      0.14       697
     neutral       0.48      0.93      0.63      2097
    negative       0.75      0.08      0.14      1819

   micro avg       0.47      0.47      0.47      4613
   macro avg       0.49      0.37      0.30      4613
weighted avg       0.55      0.47      0.36      4613

[[ 142 1549  128]
 [  36 1960  101]
 [  11  615   71]]


In [103]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()

clf.fit(X_train_nb, y_train_nb)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred, target_names=labels))

print(confusion_matrix(y_test, y_pred, labels))

              precision    recall  f1-score   support

    positive       0.00      0.00      0.00       697
     neutral       0.46      1.00      0.63      2097
    negative       0.77      0.01      0.02      1819

   micro avg       0.46      0.46      0.46      4613
   macro avg       0.41      0.34      0.21      4613
weighted avg       0.51      0.46      0.29      4613

[[  17 1802    0]
 [   5 2092    0]
 [   0  697    0]]


/home/muskan_2117/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# Using Logistic regression

#### Solver = sag 

In [86]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression(solver='sag')

classifier.fit(X_train, y_train)

/home/muskan_2117/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/muskan_2117/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='sag',
          tol=0.0001, verbose=0, warm_start=False)

In [87]:
y_pred = classifier.predict(X_test)

print(classification_report(y_test, y_pred, target_names=labels))

print(confusion_matrix(y_test, y_pred, labels))

              precision    recall  f1-score   support

    positive       0.62      0.38      0.47       697
     neutral       0.65      0.74      0.69      2097
    negative       0.70      0.69      0.70      1819

   micro avg       0.67      0.67      0.67      4613
   macro avg       0.66      0.61      0.62      4613
weighted avg       0.67      0.67      0.66      4613

[[1258  521   40]
 [ 415 1562  120]
 [ 112  321  264]]


#### Solver = newton-cg

In [88]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression(solver='newton-cg')

classifier.fit(X_train, y_train)

/home/muskan_2117/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='newton-cg',
          tol=0.0001, verbose=0, warm_start=False)

In [89]:
y_pred = classifier.predict(X_test)

print(classification_report(y_test, y_pred, target_names=labels))

print(confusion_matrix(y_test, y_pred, labels))

              precision    recall  f1-score   support

    positive       0.63      0.38      0.47       697
     neutral       0.65      0.75      0.69      2097
    negative       0.70      0.69      0.70      1819

   micro avg       0.67      0.67      0.67      4613
   macro avg       0.66      0.61      0.62      4613
weighted avg       0.67      0.67      0.66      4613

[[1260  519   40]
 [ 416 1563  118]
 [ 112  321  264]]


#### Solver = saga

In [90]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression(solver='saga')

classifier.fit(X_train, y_train)

/home/muskan_2117/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/muskan_2117/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='saga',
          tol=0.0001, verbose=0, warm_start=False)

In [91]:
y_pred = classifier.predict(X_test)

print(classification_report(y_test, y_pred, target_names=labels))

print(confusion_matrix(y_test, y_pred, labels))

              precision    recall  f1-score   support

    positive       0.63      0.38      0.47       697
     neutral       0.65      0.74      0.69      2097
    negative       0.70      0.69      0.70      1819

   micro avg       0.67      0.67      0.67      4613
   macro avg       0.66      0.60      0.62      4613
weighted avg       0.67      0.67      0.66      4613

[[1255  525   39]
 [ 419 1559  119]
 [ 111  322  264]]


#### Solver = lbfgs

In [92]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression(solver='lbfgs')

classifier.fit(X_train, y_train)

/home/muskan_2117/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/muskan_2117/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/muskan_2117/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/muskan_2117/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [93]:
y_pred = classifier.predict(X_test)

print(classification_report(y_test, y_pred, target_names=labels))

print(confusion_matrix(y_test, y_pred, labels))

              precision    recall  f1-score   support

    positive       0.62      0.38      0.47       697
     neutral       0.64      0.73      0.69      2097
    negative       0.70      0.69      0.69      1819

   micro avg       0.66      0.66      0.66      4613
   macro avg       0.65      0.60      0.62      4613
weighted avg       0.66      0.66      0.66      4613

[[1255  526   38]
 [ 437 1537  123]
 [ 108  326  263]]


### Using SVM

#### Linear Kernel

In [94]:
from sklearn import svm

clf = svm.SVC(kernel='linear', verbose = 1)

clf.fit(X_train, y_train)


[LibSVM]

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=1)

In [95]:
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred, target_names=labels))

print(confusion_matrix(y_test, y_pred, labels))


              precision    recall  f1-score   support

    positive       0.61      0.38      0.47       697
     neutral       0.63      0.76      0.69      2097
    negative       0.73      0.66      0.69      1819

   micro avg       0.67      0.67      0.67      4613
   macro avg       0.66      0.60      0.62      4613
weighted avg       0.67      0.67      0.66      4613

[[1206  569   44]
 [ 373 1600  124]
 [  79  354  264]]


#### Poly Kernel

In [96]:
clf = svm.SVC(kernel='poly', verbose = 1)
clf.fit(X_train, y_train)

[LibSVM]

/home/muskan_2117/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='poly', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=1)

In [97]:
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred, target_names=labels))

print(confusion_matrix(y_test, y_pred, labels))


              precision    recall  f1-score   support

    positive       0.00      0.00      0.00       697
     neutral       0.49      0.95      0.65      2097
    negative       0.72      0.22      0.34      1819

   micro avg       0.52      0.52      0.52      4613
   macro avg       0.40      0.39      0.33      4613
weighted avg       0.51      0.52      0.43      4613

[[ 407 1412    0]
 [ 108 1989    0]
 [  51  646    0]]


/home/muskan_2117/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


#### RBF Kernel

In [98]:
clf = svm.SVC(kernel='rbf', verbose = 1)
clf.fit(X_train, y_train)

/home/muskan_2117/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


[LibSVM]

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=1)

In [99]:
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred, target_names=labels))

print(confusion_matrix(y_test, y_pred, labels))


              precision    recall  f1-score   support

    positive       0.62      0.01      0.03       697
     neutral       0.58      0.83      0.68      2097
    negative       0.71      0.62      0.66      1819

   micro avg       0.62      0.62      0.62      4613
   macro avg       0.64      0.49      0.46      4613
weighted avg       0.64      0.62      0.58      4613

[[1127  690    2]
 [ 352 1741    4]
 [ 106  581   10]]


#### Sigmoid Kernel

In [100]:
clf = svm.SVC(kernel='sigmoid', verbose = 1)
clf.fit(X_train, y_train)

[LibSVM]

/home/muskan_2117/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='sigmoid', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=1)

In [101]:
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred, target_names=labels))

print(confusion_matrix(y_test, y_pred, labels))


              precision    recall  f1-score   support

    positive       0.50      0.00      0.01       697
     neutral       0.57      0.83      0.68      2097
    negative       0.70      0.59      0.64      1819

   micro avg       0.61      0.61      0.61      4613
   macro avg       0.59      0.47      0.44      4613
weighted avg       0.61      0.61      0.56      4613

[[1069  749    1]
 [ 347 1749    1]
 [ 116  579    2]]


### Stochastic Gradient Descent

#### loss = hinge

In [103]:
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(loss="hinge", penalty="l2", max_iter=1000)
clf.fit(X_train, y_train)

/home/muskan_2117/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=1000,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False)

In [104]:
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred, target_names=labels))

print(confusion_matrix(y_test, y_pred, labels))

              precision    recall  f1-score   support

    positive       0.65      0.34      0.45       697
     neutral       0.65      0.74      0.69      2097
    negative       0.69      0.70      0.69      1819

   micro avg       0.67      0.67      0.67      4613
   macro avg       0.66      0.59      0.61      4613
weighted avg       0.67      0.67      0.66      4613

[[1281  498   40]
 [ 458 1552   87]
 [ 130  330  237]]


#### loss = modified_huber

In [105]:
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(loss="modified_huber", penalty="l2", max_iter=1000)
clf.fit(X_train, y_train)

/home/muskan_2117/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='modified_huber',
       max_iter=1000, n_iter=None, n_iter_no_change=5, n_jobs=None,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       tol=None, validation_fraction=0.1, verbose=0, warm_start=False)

In [106]:
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred, target_names=labels))

print(confusion_matrix(y_test, y_pred, labels))

              precision    recall  f1-score   support

    positive       0.64      0.34      0.45       697
     neutral       0.63      0.78      0.70      2097
    negative       0.73      0.65      0.69      1819

   micro avg       0.66      0.66      0.66      4613
   macro avg       0.66      0.59      0.61      4613
weighted avg       0.67      0.66      0.66      4613

[[1187  592   40]
 [ 359 1641   97]
 [  87  371  239]]


#### loss = log

In [107]:
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(loss="log", penalty="l2", max_iter=1000)
clf.fit(X_train, y_train)

/home/muskan_2117/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=1000,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False)

In [108]:
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred, target_names=labels))

print(confusion_matrix(y_test, y_pred, labels))

              precision    recall  f1-score   support

    positive       0.63      0.35      0.45       697
     neutral       0.64      0.75      0.69      2097
    negative       0.70      0.68      0.69      1819

   micro avg       0.66      0.66      0.66      4613
   macro avg       0.66      0.59      0.61      4613
weighted avg       0.66      0.66      0.66      4613

[[1240  544   35]
 [ 416 1573  108]
 [ 107  344  246]]


### Adaboost

In [16]:
from sklearn.ensemble import AdaBoostClassifier

adbc = AdaBoostClassifier()

adbc.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None)

In [19]:
y_pred = adbc.predict(X_test)

print(classification_report(y_test, y_pred, target_names=labels))

print(confusion_matrix(y_test, y_pred, labels)) 

              precision    recall  f1-score   support

    positive       0.44      0.31      0.37       697
     neutral       0.58      0.64      0.61      2097
    negative       0.61      0.60      0.60      1819

   micro avg       0.58      0.58      0.58      4613
   macro avg       0.54      0.52      0.53      4613
weighted avg       0.57      0.58      0.57      4613

[[1092  643   84]
 [ 562 1347  188]
 [ 146  334  217]]


## Pre-processing using NLTK

#### Tokenization

In [15]:
from nltk.tokenize import word_tokenize

nltk_tokens = []
NaNCount = 0

for i in range(len(all_tweets)):
    if all_tweets.text[i] == 'Not Available':
        nltk_tokens.append(['NaN'])
        NaNCount = NaNCount + 1
    else:
        tmp = word_tokenize(str(all_tweets.text[i]))
        nltk_tokens.append(tmp) 

In [16]:
all_tweets['nltk_tokens'] = nltk_tokens 

In [17]:
import numpy as np

data = np.array([np.arange(len(all_tweets) - NaNCount)]*2).T
nltk_tweets = pd.DataFrame(data, columns = ['label','nltk_tokens']) 

j = 0

for i in range(len(all_tweets)):
    if all_tweets.nltk_tokens[i] == ['NaN']:
        continue
    else:
        nltk_tweets['label'][j] = all_tweets['label'][i]
        nltk_tweets['nltk_tokens'][j] = all_tweets['nltk_tokens'][i]
        j = j + 1

/home/muskan_2117/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/home/muskan_2117/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [114]:
nltk_tweets.head(10)

,label,nltk_tokens
0,neutral,"[Won, the, match, #, getin, ., Plus, ,, tomorr..."
1,neutral,"[Some, areas, of, New, England, could, see, th..."
2,positive,"[@, mariakaykay, aga, tayo, tomorrow, ah, ., :..."
3,neutral,"[Tina, Fey, &, amp, ;, Amy, Poehler, are, host..."
4,positive,"[Lunch, from, my, new, Lil, spot, ..., THE, CO..."
5,positive,"[SNC, Halloween, Pr, ., Pumped, ., Let, 's, wo..."
6,negative,"[@, jacquelinemegan, I, 'm, sorry, ,, I, Heart..."
7,neutral,"[Manchester, United, will, try, to, return, to..."
8,neutral,"[Going, to, a, bulls, game, with, Aaliyah, &, ..."
9,neutral,"[Any, Toon, Fans, with, a, spare, ticket, for,..."


#### Using Glove embeddings

In [18]:
nltk_embeddings = []

for i in range(len(tweets)):
    token = nltk_tweets['nltk_tokens'][i]
    embeddings = []
    for j in range(len(token)):
        try:
            embeddings.append(model[token[j]])
        except KeyError:
            continue
    nltk_embeddings.append(embeddings)            

In [19]:
nltk_wordvec_arrays = np.zeros((len(nltk_tweets), 300))

for i in range(len(nltk_tweets)):
    nltk_wordvec_arrays[i,:] = word_vector(nltk_tweets['nltk_tokens'][i], 300)
    
nltk_wordvec_df = pd.DataFrame(nltk_wordvec_arrays)
nltk_wordvec_df.shape

(41935, 300)

In [20]:
X_train_nltk, X_test_nltk, y_train_nltk, y_test_nltk = train_test_split(nltk_wordvec_df, y, test_size=0.11, random_state=42)

In [119]:
X_train_nltk.shape

(37322, 300)

In [120]:
X_test_nltk.shape

(4613, 300)

### NB

In [95]:
y_nb = tweets.label

X_train_nb, X_test_nb, y_train_nb, y_test_nb = train_test_split(nb_wordvec_df_nltk, y_nb, test_size=0.11, random_state=42)

clf = ComplementNB()

clf.fit(X_train_nb, y_train_nb)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred, target_names=labels))

print(confusion_matrix(y_test, y_pred, labels))

              precision    recall  f1-score   support

    positive       0.15      0.53      0.23       697
     neutral       0.00      0.00      0.00      2097
    negative       0.40      0.48      0.44      1819

   micro avg       0.27      0.27      0.27      4613
   macro avg       0.18      0.33      0.22      4613
weighted avg       0.18      0.27      0.21      4613

[[ 868    0  951]
 [ 968    0 1129]
 [ 331    0  366]]


/home/muskan_2117/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [96]:
clf = MultinomialNB()

clf.fit(X_train_nb, y_train_nb)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred, target_names=labels))

print(confusion_matrix(y_test, y_pred, labels))

              precision    recall  f1-score   support

    positive       0.00      0.00      0.00       697
     neutral       0.45      1.00      0.63      2097
    negative       0.00      0.00      0.00      1819

   micro avg       0.45      0.45      0.45      4613
   macro avg       0.15      0.33      0.21      4613
weighted avg       0.21      0.45      0.28      4613

[[   0 1819    0]
 [   0 2097    0]
 [   0  697    0]]


/home/muskan_2117/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### SVM

In [ ]:
clf = svm.SVC(kernel='linear')
clf.fit(X_train_nltk, y_train_nltk)
y_pred_nltk = clf.predict(X_test_nltk)

print(classification_report(y_test_nltk, y_pred_nltk, target_names=labels))

print(confusion_matrix(y_test_nltk, y_pred_nltk, labels))

              precision    recall  f1-score   support

    positive       0.59      0.28      0.37       697
     neutral       0.60      0.78      0.67      2097
    negative       0.70      0.60      0.65      1819

   micro avg       0.63      0.63      0.63      4613
   macro avg       0.63      0.55      0.57      4613
weighted avg       0.64      0.63      0.62      4613

[[1093  685   41]
 [ 372 1630   95]
 [  86  419  192]]


In [40]:
from sklearn import svm
clf = svm.SVC(kernel='poly')
clf.fit(X_train_nltk, y_train_nltk)
y_pred_nltk = clf.predict(X_test_nltk)

print(classification_report(y_test_nltk, y_pred_nltk, target_names=labels))

print(confusion_matrix(y_test_nltk, y_pred_nltk, labels))

/home/muskan_2117/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


              precision    recall  f1-score   support

    positive       0.00      0.00      0.00       697
     neutral       0.48      0.96      0.64      2097
    negative       0.76      0.17      0.27      1819

   micro avg       0.50      0.50      0.50      4613
   macro avg       0.41      0.38      0.30      4613
weighted avg       0.52      0.50      0.40      4613

[[ 304 1515    0]
 [  82 2015    0]
 [  14  683    0]]


/home/muskan_2117/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [41]:
clf = svm.SVC(kernel='rbf')
clf.fit(X_train_nltk, y_train_nltk)
y_pred_nltk = clf.predict(X_test_nltk)

print(classification_report(y_test_nltk, y_pred_nltk, target_names=labels))

print(confusion_matrix(y_test_nltk, y_pred_nltk, labels))

/home/muskan_2117/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


              precision    recall  f1-score   support

    positive       0.53      0.01      0.03       697
     neutral       0.56      0.84      0.67      2097
    negative       0.70      0.56      0.62      1819

   micro avg       0.61      0.61      0.61      4613
   macro avg       0.60      0.47      0.44      4613
weighted avg       0.61      0.61      0.56      4613

[[1017  798    4]
 [ 324 1768    5]
 [ 107  580   10]]


In [42]:
clf = svm.SVC(kernel='sigmoid')
clf.fit(X_train_nltk, y_train_nltk)
y_pred_nltk = clf.predict(X_test_nltk)

print(classification_report(y_test_nltk, y_pred_nltk, target_names=labels))

print(confusion_matrix(y_test_nltk, y_pred_nltk, labels))

/home/muskan_2117/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


              precision    recall  f1-score   support

    positive       0.33      0.01      0.01       697
     neutral       0.55      0.86      0.67      2097
    negative       0.70      0.51      0.59      1819

   micro avg       0.59      0.59      0.59      4613
   macro avg       0.53      0.46      0.42      4613
weighted avg       0.58      0.59      0.54      4613

[[ 932  884    3]
 [ 294 1798    5]
 [ 103  590    4]]


In [30]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression(solver='sag')

classifier.fit(X_train_nltk, y_train_nltk)

y_pred_nltk = classifier.predict(X_test_nltk)

print(classification_report(y_test_nltk, y_pred_nltk, target_names=labels))

print(confusion_matrix(y_test_nltk, y_pred_nltk, labels))

/home/muskan_2117/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/muskan_2117/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


              precision    recall  f1-score   support

    positive       0.60      0.32      0.42       697
     neutral       0.61      0.74      0.67      2097
    negative       0.68      0.64      0.66      1819

   micro avg       0.64      0.64      0.64      4613
   macro avg       0.63      0.57      0.58      4613
weighted avg       0.64      0.64      0.63      4613

[[1161  617   41]
 [ 443 1550  104]
 [ 115  360  222]]


In [31]:
classifier = LogisticRegression(solver='newton-cg')

classifier.fit(X_train_nltk, y_train_nltk)

y_pred_nltk = classifier.predict(X_test_nltk)

print(classification_report(y_test_nltk, y_pred_nltk, target_names=labels))

print(confusion_matrix(y_test_nltk, y_pred_nltk, labels))

/home/muskan_2117/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


              precision    recall  f1-score   support

    positive       0.60      0.32      0.42       697
     neutral       0.61      0.74      0.67      2097
    negative       0.68      0.64      0.66      1819

   micro avg       0.64      0.64      0.64      4613
   macro avg       0.63      0.57      0.58      4613
weighted avg       0.64      0.64      0.63      4613

[[1160  618   41]
 [ 441 1551  105]
 [ 114  360  223]]


In [32]:
classifier = LogisticRegression(solver='saga')

classifier.fit(X_train_nltk, y_train_nltk)

y_pred_nltk = classifier.predict(X_test_nltk)

print(classification_report(y_test_nltk, y_pred_nltk, target_names=labels))

print(confusion_matrix(y_test_nltk, y_pred_nltk, labels))

/home/muskan_2117/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/muskan_2117/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


              precision    recall  f1-score   support

    positive       0.61      0.32      0.42       697
     neutral       0.61      0.74      0.67      2097
    negative       0.68      0.64      0.66      1819

   micro avg       0.64      0.64      0.64      4613
   macro avg       0.63      0.57      0.58      4613
weighted avg       0.64      0.64      0.63      4613

[[1163  616   40]
 [ 442 1551  104]
 [ 116  360  221]]


In [33]:
classifier = LogisticRegression(solver='lbfgs')

classifier.fit(X_train_nltk, y_train_nltk)

y_pred_nltk = classifier.predict(X_test_nltk)

print(classification_report(y_test_nltk, y_pred_nltk, target_names=labels))

print(confusion_matrix(y_test_nltk, y_pred_nltk, labels))

/home/muskan_2117/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/muskan_2117/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/muskan_2117/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


              precision    recall  f1-score   support

    positive       0.61      0.32      0.42       697
     neutral       0.62      0.74      0.67      2097
    negative       0.68      0.64      0.66      1819

   micro avg       0.64      0.64      0.64      4613
   macro avg       0.63      0.57      0.58      4613
weighted avg       0.64      0.64      0.63      4613

[[1166  616   37]
 [ 439 1552  106]
 [ 118  355  224]]


/home/muskan_2117/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [35]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(loss="hinge", penalty="l2", max_iter=1000)
clf.fit(X_train_nltk, y_train_nltk)
y_pred_nltk = clf.predict(X_test_nltk)
print(classification_report(y_test_nltk, y_pred_nltk, target_names=labels))
print(confusion_matrix(y_test_nltk, y_pred_nltk, labels))

/home/muskan_2117/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


              precision    recall  f1-score   support

    positive       0.74      0.12      0.21       697
     neutral       0.61      0.75      0.67      2097
    negative       0.65      0.67      0.66      1819

   micro avg       0.63      0.63      0.63      4613
   macro avg       0.66      0.52      0.51      4613
weighted avg       0.64      0.63      0.60      4613

[[1225  586    8]
 [ 493 1582   22]
 [ 172  440   85]]


In [36]:
clf = SGDClassifier(loss="modified_huber", penalty="l2", max_iter=1000)
clf.fit(X_train_nltk, y_train_nltk)
y_pred_nltk = clf.predict(X_test_nltk)
print(classification_report(y_test_nltk, y_pred_nltk, target_names=labels))
print(confusion_matrix(y_test_nltk, y_pred_nltk, labels))

/home/muskan_2117/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


              precision    recall  f1-score   support

    positive       0.61      0.28      0.38       697
     neutral       0.63      0.70      0.66      2097
    negative       0.64      0.70      0.67      1819

   micro avg       0.63      0.63      0.63      4613
   macro avg       0.63      0.56      0.57      4613
weighted avg       0.63      0.63      0.62      4613

[[1272  513   34]
 [ 550 1458   89]
 [ 166  337  194]]


In [37]:
clf = SGDClassifier(loss="log", penalty="l2", max_iter=1000)
clf.fit(X_train_nltk, y_train_nltk)
y_pred_nltk = clf.predict(X_test_nltk)
print(classification_report(y_test_nltk, y_pred_nltk, target_names=labels))
print(confusion_matrix(y_test_nltk, y_pred_nltk, labels))

/home/muskan_2117/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


              precision    recall  f1-score   support

    positive       0.61      0.28      0.39       697
     neutral       0.61      0.75      0.67      2097
    negative       0.68      0.64      0.66      1819

   micro avg       0.64      0.64      0.64      4613
   macro avg       0.63      0.56      0.57      4613
weighted avg       0.64      0.64      0.62      4613

[[1167  618   34]
 [ 439 1568   90]
 [ 121  378  198]]


In [38]:
adbc = AdaBoostClassifier()

adbc.fit(X_train_nltk, y_train_nltk)

y_pred_nltk = adbc.predict(X_test_nltk)

print(classification_report(y_test_nltk, y_pred_nltk, target_names=labels))
print(confusion_matrix(y_test_nltk, y_pred_nltk, labels))

              precision    recall  f1-score   support

    positive       0.42      0.23      0.30       697
     neutral       0.57      0.68      0.62      2097
    negative       0.60      0.57      0.59      1819

   micro avg       0.57      0.57      0.57      4613
   macro avg       0.53      0.49      0.50      4613
weighted avg       0.56      0.57      0.56      4613

[[1037  703   79]
 [ 524 1433  140]
 [ 163  375  159]]


### ekphrasis vs nltk

In [23]:
tweets['tokens'][2]

['<user>',
 'aga',
 'tayo',
 'tomorrow',
 'ah',
 '.',
 '<happy>',
 'good',
 'night',
 ',',
 'ces',
 '.',
 'love',
 'you',
 '!',
 '<devil>',
 '<']

In [24]:
nltk_tweets['nltk_tokens'][2]

['@',
 'mariakaykay',
 'aga',
 'tayo',
 'tomorrow',
 'ah',
 '.',
 ':',
 ')',
 'Good',
 'night',
 ',',
 'Ces',
 '.',
 'Love',
 'you',
 '!',
 '&',
 'gt',
 ';',
 ':',
 'D',
 '&',
 'lt',
 ';']

In [25]:
model['<happy>']

array([ 8.748700e-02,  1.497360e-01,  5.326670e-01, -1.560000e-01,
       -5.435400e-02, -7.420520e-01,  3.741750e-01, -3.096200e-02,
       -4.451240e-01,  2.979030e-01,  1.190787e+00, -2.323090e-01,
        2.707480e-01,  3.577600e-02, -2.784010e-01, -2.966300e-02,
        2.873500e-01,  5.789290e-01, -1.085148e+00,  4.069050e-01,
       -7.866730e-01, -1.962260e-01, -2.119950e-01, -1.212554e+00,
        3.784580e-01,  4.365000e-02, -3.278020e-01,  4.973920e-01,
        6.721110e-01,  8.218180e-01,  6.863920e-01, -5.409120e-01,
       -6.702300e-02,  6.368500e-02, -3.590410e-01,  1.859400e-01,
       -2.181380e-01,  3.023610e-01, -6.591900e-02,  4.849880e-01,
        8.560620e-01,  1.249526e+00, -1.777450e-01,  7.323290e-01,
        4.834900e-01,  2.590170e-01, -1.078250e-01,  7.514200e-02,
        7.747760e-01,  3.823860e-01, -9.880800e-02, -7.663040e-01,
       -2.038410e-01,  1.536300e-01,  1.214650e-01, -1.225600e-01,
        4.677220e-01,  4.571680e-01,  4.447220e-01, -4.920670e

In [26]:
model[':)']

array([ 0.116521,  0.15463 ,  0.047883,  0.063743,  0.112395, -0.065712,
       -0.044426, -0.011419,  0.098808,  0.042539, -0.110289,  0.019689,
       -0.009833,  0.028707, -0.081217, -0.081131, -0.234737, -0.071947,
        0.030969,  0.135651,  0.056755, -0.148565, -0.055295, -0.180719,
       -0.070928, -0.029374, -0.022661, -0.001052,  0.022075, -0.074194,
       -0.091624,  0.099024,  0.241266, -0.003809, -0.04262 , -0.24047 ,
        0.01284 , -0.184066, -0.04188 , -0.040969,  0.136735, -0.16749 ,
        0.015138, -0.257993,  0.045962, -0.268149,  0.030862, -0.092176,
        0.122345,  0.030217, -0.160942,  0.011762, -0.119691,  0.001137,
       -0.01738 ,  0.117686, -0.182717,  0.065644, -0.082618,  0.022352,
       -0.164335,  0.145822,  0.041732, -0.074911,  0.218303,  0.069315,
        0.057877,  0.024107, -0.066434, -0.064211,  0.110319, -0.115906,
       -0.043664,  0.14885 ,  0.02675 ,  0.09146 ,  0.05057 , -0.180723,
       -0.064295, -0.083177, -0.008054,  0.10666 , 